In [145]:
import xml.etree.ElementTree as ET
import numpy as np
import re
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [266]:
#file = ET.parse('/Users/s0c02nj/Desktop/Brainwaves/ML/data/17_bk.xml')
df = pd.read_csv('/Users/s0c02nj/Desktop/Brainwaves/ML/train_labels.csv')
files = df.File_Name
NSMAP = {'ab':'http://www.abbyy.com/FineReader_xml/FineReader10-schema-v1.xml'}

In [285]:
df['party_A'].mode()

0    SOCIETE GENERALE
dtype: object

In [286]:
df_test = pd.read_csv('/Users/s0c02nj/Desktop/Brainwaves/submission_format6c29943.csv')
files2 = df_test.File_Name

In [287]:
#pages_text[1][0]----Paragraph
def read_xml(xml_file):
    xml_file = '/Users/s0c02nj/Desktop/Brainwaves/private_test_data/' + xml_file + '.xml'
    tree = ET.parse(xml_file) 
    root = tree.getroot()
    pages_text = []
    
    for page in root.findall('.//ab:page', namespaces=NSMAP):

        paras = []
        temp = ""
        for block in page.findall('ab:block', namespaces=NSMAP):
            for line in block.findall('ab:text/ab:par/ab:line/ab:formatting', namespaces=NSMAP):
                temp = temp + " " + line.text

            paras.append(temp)
            temp = ""

        pages_text.append(paras)
    
    return pages_text

In [288]:
def get_relevent_ind(l):
    inds = []
    for i,e in enumerate(l):
        #e = re.sub(r'\"','',e)
        if 'calculation agent' in e.lower() :
            #print (e)
            #print ('Hello')
            #if i > 0 and i < len(l)-1: 
                #for j in range(1,30):
            inds.append(i)
            inds.append(i+1)
            inds.append(i+2)
            inds.append(i+3)
            inds.append(i+4)
            inds.append(i+5)
            inds.append(i+6)
            inds.append(i+7)
            inds.append(i+8)
            inds.append(i+9)
            inds.append(i+10)
            inds.append(i+11)
            inds.append(i+12)
            inds.append(i+13)
            inds.append(i+14)
            inds.append(i+15)
            inds.append(i+16)
            inds.append(i+17)
            inds.append(i+18)
            inds.append(i+19)
            inds.append(i+20)
                
    return(list(set(inds)))

In [289]:
def get_term_curr_para(file):
    list_rel_doc =[]
    page_text = read_xml(file)
    for page in page_text:
        inds = get_relevent_ind(page)
        l = len(page) 
        page = " ".join([page[i] for i in inds if i>=0 and i<l])
        if page != '':
            list_rel_doc.append(page)
    
    return list_rel_doc
        

In [290]:
def get_most_rel(file):
    doc1=''
    list1=[]
    doc = get_term_curr_para(file)
    
    if len(doc)>1:
        for i in range(0,len(doc)):
            doc1 = doc[i]+doc1
        list1.append(doc1)
    
    elif len(doc)==1:
        list1.append(doc[0])
    
    return list1

In [250]:
paras = []
for i,file in enumerate(files2):
    #print(i)
    page = get_most_rel(file)
    paras.append(page)

In [251]:
len(paras)

61

In [252]:
txt=[]
for i in range(0,len(paras)):
    if len(paras[i])!=0: 
        #print (i)
        txt.append(paras[i][0])
    else:
        txt.append('')

In [253]:
len(txt)

61

#### Model Fitting for Diefault Dispute

In [254]:
df_test['text_sum']=txt

In [133]:
df1 = df[['text_sum','CA_fallback_default_dispute','CA_dispute_resolution ']]

In [255]:
df_test['text_sum'] = df_test['text_sum'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
df_test['text_sum'] = df_test['text_sum'] .apply(lambda x:x.lower())

In [242]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english',max_df=1.0, min_df=1, max_features=600)
vectorizer.fit_transform(df1['text_sum'])
tf_feat = vectorizer.transform(df1['text_sum'])

In [243]:
from scipy.sparse import csr_matrix
tf_idf_arr= tf_feat.toarray()

In [244]:
#Response 
le= LabelEncoder()
y_cat=le.fit_transform(df1['CA_fallback_default_dispute'])

In [245]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tf_idf_arr,y_cat,test_size=0.5, random_state=42)

In [246]:
from sklearn.ensemble import RandomForestClassifier

rf =  RandomForestClassifier(bootstrap=True, criterion='gini',
                max_depth=3, max_features='auto', max_leaf_nodes=None, 
                min_impurity_decrease=0.0, min_impurity_split=None,
                min_samples_leaf=1, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
                oob_score=False, random_state=0, verbose=0, warm_start=False)

In [247]:
rf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [248]:
rf.score(x_train,y_train)

0.8225806451612904

In [249]:
rf.score(x_test,y_test)

0.84

In [256]:
x_test = vectorizer.transform(df_test['text_sum'])

In [257]:
test_pred = rf.predict(x_test)

In [260]:
#le.inverse_transform(test_pred)

#### Predicting dispute resolution

In [215]:
#Response 
le2= LabelEncoder()
y_cat2=le2.fit_transform(df['CA_fallback_default_dispute'])

In [216]:
from sklearn.model_selection import train_test_split
x1_train,x1_test,y1_train,y1_test = train_test_split(tf_idf_arr,y_cat2,test_size=0.1, random_state=42)

In [214]:
#df['CA_fallback_default_dispute'].value_counts()

In [224]:
from sklearn.ensemble import RandomForestClassifier

rf2 =  RandomForestClassifier(bootstrap=True, criterion='gini',
                max_depth=3, max_features='auto', max_leaf_nodes=None, 
                min_impurity_decrease=0.0, min_impurity_split=None,
                min_samples_leaf=1, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
                oob_score=False, random_state=0, verbose=0, warm_start=False)

In [225]:
rf2.fit(x1_train,y1_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [226]:
rf2.score(x1_train,y1_train)

0.8348214285714286

In [227]:
rf2.score(x1_test,y1_test)

0.76

In [277]:
#le.inverse_transform(rf2.predict(x_test))
df_test['CA_fallback_default_dispute'] = p
df_test['CA_dispute_resolution ']=p

In [278]:
df_test.to_csv('/Users/s0c02nj/Desktop/test_v4.csv')